<a href="https://colab.research.google.com/github/diwashsapkota/movierecommendationsystem/blob/main/collaborativefiltering2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import  drive
drive.mount('/drive')


Mounted at /drive


In [15]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# moviesurl = 'https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Collaborative%20Filtering/dataset/movies.csv'
# linksurl = 'https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Collaborative%20Filtering/dataset/links.csv'
# tagsurl = 'https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Collaborative%20Filtering/dataset/tags.csv'
# ratingsurl = 'https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Collaborative%20Filtering/dataset/ratings.csv'
# toyurl = 'https://raw.githubusercontent.com/codeheroku/Introduction-to-Machine-Learning/master/Collaborative%20Filtering/dataset/toy_dataset.csv'

In [17]:
# import requests
# movies = pd.read_csv(moviesurl)
# movies.to_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/movies.csv')

In [18]:
# links = pd.read_csv(linksurl)
# links.to_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/links.csv')

In [19]:
# tags = pd.read_csv(tagsurl)
# tags.to_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/tags.csv')

In [20]:
# ratings = pd.read_csv(ratingsurl)
# ratings.to_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/ratings.csv')

In [21]:
# toy_dataset = pd.read_csv(toyurl, index_col = 0)
# toy_dataset.to_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/toy_dataset.csv')

In [50]:
movies = pd.read_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/movies.csv')
links = pd.read_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/links.csv')
tags = pd.read_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/tags.csv')
ratings = pd.read_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/ratings.csv')
toy_dataset = pd.read_csv('/drive/My Drive/Colab Notebooks/code heroku/collaborativefiltering/toy_dataset.csv', index_col = 0)

In [51]:
toy_dataset.head()

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,NaN,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,NaN
user 3,1.0,NaN,NaN,4.0,5.0,4.0
user 4,NaN,2.0,1.0,4.0,NaN,3.0
user 5,1.0,NaN,2.0,3.0,3.0,4.0


In [52]:
toy_dataset.fillna(0, inplace = True)
toy_dataset.head()

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,4.0,5.0,3.0,0.0,2.0,1.0
user 2,5.0,3.0,3.0,2.0,2.0,0.0
user 3,1.0,0.0,0.0,4.0,5.0,4.0
user 4,0.0,2.0,1.0,4.0,0.0,3.0
user 5,1.0,0.0,2.0,3.0,3.0,4.0


In [53]:
def standardize(row):
    new_row = (row - row.mean()) / (row.max() - row.min())
    return new_row
    
toy_dataset_std = toy_dataset.apply(standardize)
toy_dataset_std

,action1,action2,action3,romantic1,romantic2,romantic3
user 1,0.36,0.6,0.400000,-0.65,-0.08,-0.35
user 2,0.56,0.2,0.400000,-0.15,-0.08,-0.60
user 3,-0.24,-0.4,-0.600000,0.35,0.52,0.40
user 4,-0.44,0.0,-0.266667,0.35,-0.48,0.15
user 5,-0.24,-0.4,0.066667,0.10,0.12,0.40


In [54]:
#We are taking a transpose since we want similarity between items which need to be in rows
item_similarity = cosine_similarity(toy_dataset_std.T)
print(item_similarity)

[[ 1.          0.70668875  0.81368151 -0.79941088 -0.02539184 -0.91410609]
 [ 0.70668875  1.          0.72310153 -0.84515425 -0.5189993  -0.84337386]
 [ 0.81368151  0.72310153  1.         -0.84794611 -0.3799803  -0.80218063]
 [-0.79941088 -0.84515425 -0.84794611  1.          0.14803913  0.72374686]
 [-0.02539184 -0.5189993  -0.3799803   0.14803913  1.          0.39393939]
 [-0.91410609 -0.84337386 -0.80218063  0.72374686  0.39393939  1.        ]]


In [55]:
item_similarity_df = pd.DataFrame(item_similarity, index = toy_dataset.columns, columns = toy_dataset.columns)
item_similarity_df

,action1,action2,action3,romantic1,romantic2,romantic3
action1,1.000000,0.706689,0.813682,-0.799411,-0.025392,-0.914106
action2,0.706689,1.000000,0.723102,-0.845154,-0.518999,-0.843374
action3,0.813682,0.723102,1.000000,-0.847946,-0.379980,-0.802181
romantic1,-0.799411,-0.845154,-0.847946,1.000000,0.148039,0.723747
romantic2,-0.025392,-0.518999,-0.379980,0.148039,1.000000,0.393939
romantic3,-0.914106,-0.843374,-0.802181,0.723747,0.393939,1.000000


In [56]:
#Let's make recommendations
def get_similar_movies(movie_name, user_rating):
    similar_score = item_similarity_df[movie_name] * user_rating
    similar_score = similar_score.sort_values(ascending = False)
    return similar_score

print(get_similar_movies("action1", 5))

action1      5.000000
action3      4.068408
action2      3.533444
romantic2   -0.126959
romantic1   -3.997054
romantic3   -4.570530
Name: action1, dtype: float64


In [60]:
action_lover =  [('action1', 5), ('romantic2', 1), ('romantic3', 1)]
similar_movies = pd.DataFrame()
for movie, rating in action_lover:
    similar_movies = similar_movies.append(get_similar_movies(movie, rating), ignore_index = True)

print(similar_movies.head())
similar_movies.sum().sort_values(ascending = False)

    action1   action2   action3  romantic1  romantic2  romantic3
0  5.000000  3.533444  4.068408  -3.997054  -0.126959  -4.570530
1 -0.025392 -0.518999 -0.379980   0.148039   1.000000   0.393939
2 -0.914106 -0.843374 -0.802181   0.723747   0.393939   1.000000


action1      4.060502
action3      2.886247
action2      2.171071
romantic2    1.266980
romantic1   -3.125268
romantic3   -3.176591
dtype: float64